In [42]:
import yfinance as yf
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import DataLoader, TensorDataset
import sys

def fetch_data(ticker, start, end):
    df = yf.download(ticker, start=start, end=end)
    df.dropna(inplace=True)
    return df

def compute_technical_indicators(df):
    # Simple Moving Averages
    df["SMA10"] = df["Adj Close"].rolling(window=10).mean()
    df["SMA20"] = df["Adj Close"].rolling(window=20).mean()
    df["SMA50"] = df["Adj Close"].rolling(window=50).mean()
    df["SMA100"] = df["Adj Close"].rolling(window=100).mean()
    # Exponential Moving Averages
    df["EMA12"] = df["Adj Close"].ewm(span=12, adjust=False).mean()
    df["EMA26"] = df["Adj Close"].ewm(span=26, adjust=False).mean()
    # RSI14
    delta = df["Adj Close"].diff()
    gain = delta.clip(lower=0)
    loss = -delta.clip(upper=0)
    avg_gain = gain.rolling(window=14).mean()
    avg_loss = loss.rolling(window=14).mean()
    rs = avg_gain / avg_loss
    df["RSI14"] = 100 - (100 / (1 + rs))
    # MACD, Signal, Histogram
    df["MACD"] = df["EMA12"] - df["EMA26"]
    df["MACD_Signal"] = df["MACD"].ewm(span=9, adjust=False).mean()
    df["MACD_Hist"] = df["MACD"] - df["MACD_Signal"]
    # Momentum10
    df["Momentum10"] = df["Adj Close"].pct_change(periods=10)
    # ATR14
    high_low = df["High"] - df["Low"]
    high_close = (df["High"] - df["Adj Close"].shift()).abs()
    low_close = (df["Low"] - df["Adj Close"].shift()).abs()
    df["TR"] = pd.concat([high_low, high_close, low_close], axis=1).max(axis=1)
    df["ATR14"] = df["TR"].rolling(window=14).mean()
    # Bollinger Bands (using SMA20)
    df["BB_Upper"] = df["SMA20"] + 2 * df["Adj Close"].rolling(window=20).std()
    df["BB_Lower"] = df["SMA20"] - 2 * df["Adj Close"].rolling(window=20).std()
    df["BB_Width"] = (df["BB_Upper"] - df["BB_Lower"]) / df["SMA20"]
    # Stochastic Oscillator %K and %D (14, 3)
    df["Stoch_%K"] = 100 * ((df["Adj Close"] - df["Low"].rolling(window=14).min()) /
                            (df["High"].rolling(window=14).max() - df["Low"].rolling(window=14).min()))
    df["Stoch_%D"] = df["Stoch_%K"].rolling(window=3).mean()
    # Commodity Channel Index (CCI)
    df["Typical_Price"] = (df["High"] + df["Low"] + df["Adj Close"]) / 3
    df["CCI"] = (df["Typical_Price"] - df["Typical_Price"].rolling(window=20).mean()) / (0.015 * df["Typical_Price"].rolling(window=20).std())
    # Williams %R (14)
    df["Williams_%R"] = -100 * ((df["High"].rolling(window=14).max() - df["Adj Close"]) /
                                (df["High"].rolling(window=14).max() - df["Low"].rolling(window=14).min()))
    # ADX (14)
    df["UpMove"] = df["High"] - df["High"].shift(1)
    df["DownMove"] = df["Low"].shift(1) - df["Low"]
    df["+DM"] = np.where((df["UpMove"] > df["DownMove"]) & (df["UpMove"] > 0), df["UpMove"], 0)
    df["-DM"] = np.where((df["DownMove"] > df["UpMove"]) & (df["DownMove"] > 0), df["DownMove"], 0)
    df["+DI"] = 100 * (df["+DM"].rolling(window=14).sum() / df["ATR14"])
    df["-DI"] = 100 * (df["-DM"].rolling(window=14).sum() / df["ATR14"])
    df["DX"] = 100 * (abs(df["+DI"] - df["-DI"]) / (df["+DI"] + df["-DI"]))
    df["ADX"] = df["DX"].rolling(window=14).mean()
    # OBV
    df["OBV"] = (np.sign(df["Adj Close"].diff()) * df["Volume"]).fillna(0).cumsum()
    # Chaikin Money Flow (CMF) over 20 periods
    mf_multiplier = ((df["Adj Close"] - df["Low"]) - (df["High"] - df["Adj Close"])) / (df["High"] - df["Low"])
    mf_volume = mf_multiplier * df["Volume"]
    df["CMF"] = mf_volume.rolling(window=20).sum() / df["Volume"].rolling(window=20).sum()
    # Accumulation Distribution Line (ADL)
    df["ADL"] = (((df["Adj Close"] - df["Low"]) - (df["High"] - df["Adj Close"])) / (df["High"] - df["Low"])) * df["Volume"]
    df["ADL"] = df["ADL"].cumsum()
    # ROC (12)
    df["ROC12"] = df["Adj Close"].pct_change(periods=12) * 100
    # Volatility14 (std dev of returns over 14 days)
    df["Volatility14"] = df["Adj Close"].pct_change().rolling(window=14).std()
    # Ichimoku: Tenkan-sen (9) and Kijun-sen (26)
    df["Tenkan_sen"] = (df["High"].rolling(window=9).max() + df["Low"].rolling(window=9).min()) / 2
    df["Kijun_sen"] = (df["High"].rolling(window=26).max() + df["Low"].rolling(window=26).min()) / 2
    # Drop temporary columns
    df.drop(columns=["TR", "UpMove", "DownMove", "+DM", "-DM", "DX", "Typical_Price"], inplace=True)
    return df

def create_samples(data, threshold=0.005, features=None):
    if features is None:
        features = ["Adj Close", "SMA10", "SMA20", "SMA50", "SMA100", "EMA12", "EMA26", "RSI14",
                    "MACD", "MACD_Signal", "MACD_Hist", "Momentum10", "ATR14",
                    "BB_Upper", "BB_Lower", "BB_Width", "Stoch_%K", "Stoch_%D",
                    "CCI", "Williams_%R", "ADX", "OBV", "CMF", "ADL", "ROC12",
                    "Volatility14", "Tenkan_sen", "Kijun_sen"]
    X = []
    y = []
    for i in range(len(data) - 1):
        sample = data.iloc[i][features].values
        prev_close = data.iloc[i]["Adj Close"]
        next_close = data.iloc[i+1]["Adj Close"]
        change = (next_close - prev_close) / prev_close
        if change > threshold:
            label = 1
        elif change < -threshold:
            label = 0
        else:
            label = 2
        X.append(sample)
        y.append(label)
    return np.array(X), np.array(y)

def compute_buy_hold_return(data):
    return (1 + data["Return"]).cumprod()

def compute_strategy_return(data, signals):
    return (1 + signals * data["Return"]).cumprod()

def get_signals_and_losses(X, y, model, criterion, entropy_lambda):
    model.eval()
    predictions = []
    losses = []
    with torch.no_grad():
        for i in range(len(X)):
            logits = model(X[i].unsqueeze(0))
            ce = criterion(logits, y[i].unsqueeze(0))
            ent = -torch.sum(torch.softmax(logits, dim=-1) * torch.log(torch.softmax(logits, dim=-1) + 1e-8))
            loss_val = ce + entropy_lambda * ent
            losses.append(loss_val.item())
            pred = torch.argmax(logits, dim=-1).item()
            if pred == 1:
                action = 1
            elif pred == 0:
                action = -1
            else:
                action = 0
            predictions.append(action)
    return predictions, losses

def backtest_region(symbol, start, end, model, scaler, threshold=0.005, features=None):
    df = fetch_data(symbol, start, end)
    df = compute_technical_indicators(df)
    df.dropna(inplace=True)
    df["Return"] = df["Adj Close"].pct_change()
    df.dropna(inplace=True)
    if features is None:
        features = ["Adj Close", "SMA10", "SMA20", "SMA50", "SMA100", "EMA12", "EMA26", "RSI14",
                    "MACD", "MACD_Signal", "MACD_Hist", "Momentum10", "ATR14",
                    "BB_Upper", "BB_Lower", "BB_Width", "Stoch_%K", "Stoch_%D",
                    "CCI", "Williams_%R", "ADX", "OBV", "CMF", "ADL", "ROC12",
                    "Volatility14", "Tenkan_sen", "Kijun_sen"]
    df[features] = scaler.transform(df[features])
    X_np, y_np = create_samples(df, threshold=threshold, features=features)
    X_tensor = torch.tensor(X_np, dtype=torch.float32).to(device)
    y_tensor = torch.tensor(y_np, dtype=torch.long).to(device)
    signals, losses = get_signals_and_losses(X_tensor, y_tensor, model, criterion, entropy_lambda)
    df_bt = df.iloc[1:].copy()
    df_bt["Signal"] = signals
    df_bt["Correct Signal"] = y_tensor.cpu().numpy()
    df_bt["Loss"] = losses
    bh = compute_buy_hold_return(df_bt)
    strat = compute_strategy_return(df_bt, df_bt["Signal"])
    print("Buy and Hold Final Cumulative Return:", bh.iloc[-1])
    print("MLP Strategy Final Cumulative Return:", strat.iloc[-1])
    return df_bt, bh, strat

def predict_current(current_values, model):
    model.eval()
    x = torch.tensor(current_values, dtype=torch.float32).to(device)
    x = x.unsqueeze(0)
    logits = model(x)
    pred = torch.argmax(logits, dim=-1).item()
    if pred == 1:
        action = 1
    elif pred == 0:
        action = -1
    else:
        action = 0
    return action

In [ ]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
symbol = "AAPL"
train_start = "2008-01-01"
train_end = "2023-01-01"
df_train = fetch_data(symbol, train_start, train_end)
df_train = compute_technical_indicators(df_train)
df_train.dropna(inplace=True)
df_train["Return"] = df_train["Adj Close"].pct_change()
df_train.dropna(inplace=True)
features = ["Adj Close", "SMA10", "SMA20", "SMA50", "SMA100", "EMA12", "EMA26", "RSI14",
            "MACD", "MACD_Signal", "MACD_Hist", "Momentum10", "ATR14",
            "BB_Upper", "BB_Lower", "BB_Width", "Stoch_%K", "Stoch_%D",
            "CCI", "Williams_%R", "ADX", "OBV", "CMF", "ADL", "ROC12",
            "Volatility14", "Tenkan_sen", "Kijun_sen"]
scaler = MinMaxScaler()
df_train[features] = scaler.fit_transform(df_train[features])
X_train_np, y_train_np = create_samples(df_train, threshold=0.005, features=features)
X_train = torch.tensor(X_train_np, dtype=torch.float32).to(device)
y_train = torch.tensor(y_train_np, dtype=torch.long).to(device)
train_dataset = TensorDataset(X_train, y_train)
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)

/Users/yohaanchokhany/anaconda3/lib/python3.10/site-packages/yfinance/utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
[*********************100%%**********************]  1 of 1 completed
/var/folders/k4/x9r5q9n17yj4bbwp1mnkbt_c0000gn/T/ipykernel_27191/670288794.py:101: RuntimeWarning: divide by zero encountered in double_scalars
  change = (next_close - prev_close) / prev_close


In [3]:
input_size = len(features)
hidden_size = 128
num_classes = 3
dropout_rate = 0.2

# Build MLP using PyTorch (with dropout)
model = nn.Sequential(
    nn.Linear(input_size, hidden_size),
    nn.ReLU(),
    nn.Dropout(dropout_rate),
    nn.Linear(hidden_size, num_classes)
).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
entropy_lambda = 0.05
reg_lambda = 0.001

In [4]:
print(f"Number of parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad)}")

Number of parameters: 4099


In [5]:
epochs = 200
for epoch in range(epochs):
    total_batches = len(train_dataloader)
    rolling_count = 0
    rolling_loss = 0
    for batch_idx, (batch_X, batch_y) in enumerate(train_dataloader):
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)
        optimizer.zero_grad()
        logits = model(batch_X)
        ce_loss = criterion(logits, batch_y)
        ent = -torch.sum(torch.softmax(logits, dim=-1) * torch.log(torch.softmax(logits, dim=-1) + 1e-8)) / batch_X.size(0)
        reg_loss = sum(torch.sum(param**2) for param in model.parameters())
        loss = ce_loss + entropy_lambda * ent + reg_lambda * reg_loss
        loss.backward()
        optimizer.step()
        progress = (batch_idx + 1) / total_batches * 100
        rolling_loss += loss.item()
        rolling_count += 1
        sys.stdout.write(f"\rEpoch {epoch+1}/{epochs} - [{'#' * int(progress/100 * 30) + ' ' * (30 - int(progress/100 * 30))}] {progress:.2f}% Loss: {rolling_loss/rolling_count:.6f}")
        sys.stdout.flush()
    print()


Epoch 1/200 - [##############################] 100.00% Loss: 1.129909
Epoch 2/200 - [##############################] 100.00% Loss: 1.068402
Epoch 3/200 - [##############################] 100.00% Loss: 1.066089
Epoch 4/200 - [##############################] 100.00% Loss: 1.059545
Epoch 5/200 - [##############################] 100.00% Loss: 1.057152
Epoch 6/200 - [##############################] 100.00% Loss: 1.051583
Epoch 7/200 - [##############################] 100.00% Loss: 1.042790
Epoch 8/200 - [##############################] 100.00% Loss: 1.048832
Epoch 9/200 - [##############################] 100.00% Loss: 1.054544
Epoch 10/200 - [##############################] 100.00% Loss: 1.038398
Epoch 11/200 - [##############################] 100.00% Loss: 1.042988
Epoch 12/200 - [##############################] 100.00% Loss: 1.040613
Epoch 13/200 - [##############################] 100.00% Loss: 1.036957
Epoch 14/200 - [##############################] 100.00% Loss: 1.038226
Epoch 15/200 - 

In [43]:
train_df, train_bh, train_strat = backtest_region(symbol, train_start, train_end, model, scaler, threshold=0.005, features=features)

/Users/yohaanchokhany/anaconda3/lib/python3.10/site-packages/yfinance/utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
[*********************100%%**********************]  1 of 1 completed
/var/folders/k4/x9r5q9n17yj4bbwp1mnkbt_c0000gn/T/ipykernel_27191/3955149840.py:101: RuntimeWarning: divide by zero encountered in double_scalars
  change = (next_close - prev_close) / prev_close


Buy and Hold Final Cumulative Return: 1.6592256155351568
MLP Strategy Final Cumulative Return: 9.732773673177643


In [44]:
test_df, test_bh, test_strat = backtest_region(symbol, "2005-01-01", "2008-01-01", model, scaler, threshold=0.005, features=features)

/Users/yohaanchokhany/anaconda3/lib/python3.10/site-packages/yfinance/utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
[*********************100%%**********************]  1 of 1 completed


Buy and Hold Final Cumulative Return: 4.862050776188932
MLP Strategy Final Cumulative Return: 6.148050845412252
